In [20]:
import PyPDF2
import requests
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import urllib.request
import os

In [16]:
base_url = 'https://api.govinfo.gov'
query = '?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR'

In [17]:
bill_ids = []
def get_bills(offset=0):
    global bill_ids
    f = requests.get(f'{base_url}/collections/BILLS/2022-01-01T00:00:00Z{query}&offset={offset}&pageSize=100').json()
    print(f'{base_url}/collections/BILLS/2022-01-01T00:00:00Z{query}')
    f = f['packages']
    bill_ids += [bill['packageId'] for bill in f]
for x in range(100, 1100, 100):
    get_bills(offset=x)

https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSCF8QtzhR
https://api.govinfo.gov/collections/BILLS/2022-01-01T00:00:00Z?api_key=oheReJtqERpvb0Wp49qV4Exow10jobDSC

In [12]:
len(bill_ids)

1000

In [13]:
def parse_html(text):
    text = text.decode('ascii')
    text = text[17:-21]
    text = text.replace('&lt;DOC&gt;', '')
    text = text.replace('&lt;all&gt;', '')
    return text

In [14]:
def get_text(package_id):
    f = requests.get(f'{base_url}/packages/{package_id}/htm{query}').content
    return parse_html(f)

In [21]:
for bill in bill_ids:
    if f'{bill}.txt' not in os.listdir('./sample_pdfs'):
        open(f'./sample_pdfs/{bill}.txt', 'w+').write(get_text(bill))

In [13]:
def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # get first 3 pages of the pdf file
    for page in PDFPage.get_pages(pdf_file, pagenos=(0, 1, 2)):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
    return text

In [18]:
def get_document_text(pub_id, doc_id):
    response = requests.get(f'{base_url}/Publications/{pub_id}/Documents/{doc_id}/Download')
    print(response.status_code)
    with io.BytesIO(response.content) as pdf_file:
        return pdf_to_text(pdf_file)

In [21]:
publication = get_bill_publications(bill_ids[1])[0]
document = publication['links'][0]['id']
print(publication)
get_document_text(publication['id'], document)

{'house': 'Commons', 'id': 18315, 'title': 'Bill 276 2017-19 (as introduced)', 'publicationType': {'id': 5, 'name': 'Bill', 'description': 'Full text of the Bill as introduced and further versions of the Bill as it is reprinted to incorporate amendments (proposals for change) made during its passage through Parliament.'}, 'displayDate': '2018-11-20T00:00:00', 'links': [{'id': 25898, 'title': 'Bill 276 2017-19 (as introduced)', 'url': 'https://publications.parliament.uk/pa/bills/cbill/2017-2019/0276/cbill_2017-20190276_en_1.htm', 'contentType': 'text/html'}, {'id': 25899, 'title': 'Bill 276 2017-19 (as introduced)', 'url': 'https://publications.parliament.uk/pa/bills/cbill/2017-2019/0276/18276.pdf', 'contentType': 'application/pdf'}], 'files': []}
404


PDFSyntaxError: No /Root object! - Is this really a PDF?